In [3]:
import pandas as pd
import numpy as np
import PyPDF2
import spacy  # NLP library for text processing
import re
from nltk.corpus import stopwords
import nltk
from difflib import get_close_matches
import spacy
from skill_edu import education_keywords, skill_dataset
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load a pre-trained NER model (e.g., spaCy's model)
nlp = spacy.load("en_core_web_sm")

nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anneezurike/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

In [5]:
def extract_text_from_pdf(pdf_file):
    """Extracts text from a PDF resume (given a PDF file object) and cleans it."""
    if isinstance(pdf_file, str):  # If the input is a file path
        with open(pdf_file, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            text = " ".join([page.extract_text() for page in reader.pages if page.extract_text()])
    else:  # If the input is already a file object 
        reader = PyPDF2.PdfReader(pdf_file)
        text = " ".join([page.extract_text() for page in reader.pages if page.extract_text()])

    return clean_text(text)

In [53]:
def clean_text(text):
    # Check if the text is valid and non-empty
    if pd.isna(text) or text.strip() == "":
        return ""  # or return some default text
    """Cleans the extracted text by removing special characters, numbers, and stopwords."""
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

In [9]:
def to_lower(text_array):
    if text_array:
        lowercase_array = list(map(str.lower, text_array))
        return lowercase_array
    else:
        return []

In [11]:
def remove_sensitive_info(text):
    """Removes potential bias-related words from resume text."""
    bias_keywords = ["male", "female", "black", "white", "asian", "hispanic", "married", "single"]
    for word in bias_keywords:
        text = text.replace(word, " ")
    return text

In [49]:
experience_patterns = [
        r"(\d+)\s*(?:\+|-)?\s*(?:years?|yrs?)\s*(?:of)?\s*experience",  # e.g., "5 years of experience"
        r'(\d+)\s*(?:\+|-)?\s*(?:years?|yrs?)\s*(?:in|working in|as)',  # e.g., "3+ years in software development"
        r'(\d+)-(\d+)\s*years'  # e.g., "3-5 years experience"
    ]

def extract_entities(text):
    """Efficiently extracts skills, education, and experience from resume text."""
    extracted_info = {"Skills": set(), "Education": set(), "Experience": set()}

    # Convert text to lowercase for case-insensitive matching
    text = text.lower()
    skill_data = to_lower(skill_dataset)
    education_data = to_lower(education_keywords)

    # Fast skill matching using set intersection
    extracted_info["Skills"] = {skill for skill in skill_data if skill in text}

    # Fast education matching
    extracted_info["Education"] = {edu for edu in education_data if edu in text}

    # Regex-based experience extraction
    for pattern in experience_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        if matches:
            for match in matches:
                if isinstance(match, tuple):  # Handles cases like "3-5 years"
                    extracted_info["Experience"].add(f"{match[0]}-{match[1]} years")
                else:
                    extracted_info["Experience"].add(f"{match} years")

    return {
        "Skills": list(extracted_info["Skills"]),
        "Education": list(extracted_info["Education"]),
        "Experience": list(extracted_info["Experience"])
    }

In [55]:
resume = extract_text_from_pdf('./applicants_resumes/EZURIKE ANNE_CHIBUZO_CV.pdf')
print(extract_entities(resume))

found
{'Skills': ['data visualization', 'symfony', 'graphql', 'php', 'html', 'seo', 'git', 'sem', 'restful apis', 'docker', 'java', 'javascript', 'laravel', 'python', 'web development', 'css', 'mysql', 'sql'], 'Education': ['software engineering', 'computer science', 'history', 'bsc', 'web development', 'education', 'management'], 'Experience': ['8 years']}


In [13]:
# Load pre-trained BERT model for embeddings
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

In [15]:
def get_resume_ranking_score(resume_text, job_description):
    """Ranks resumes using BERT embeddings and cosine similarity."""
    # Extract and clean resume text
    cleaned_resume = remove_sensitive_info(resume_text) # remove bias

    # Encode the resume and job description
    resume_embedding = bert_model.encode(cleaned_resume)
    job_embedding = bert_model.encode(job_description)

    # Compute similarity score
    similarity_score = cosine_similarity([resume_embedding], [job_embedding])[0][0]
    
    return float(similarity_score)

In [17]:
# Example Usage
real_resume_path = "./applicants_resumes/EZURIKE ANNE_CHIBUZO_CV.pdf"
job_desc = "Our client is looking for a PHP Symfony Developer who impresses with Practical  experience in the backend development of websites and applications as well as with APIs. Good knowledge of PHP (Symfony) and MySQL as well as object-oriented programming and an understanding of database and caching systems such as MySQL, Redis and ElasticSearch are an advantage. experience in software design techniques, test-driven development and distributed architecture. excellent communication skills. Fluent written and spoken German or English"

resume_score = get_resume_ranking_score(real_resume_path, job_desc)
print(f"🔹 **Real Resume Ranking Score:** {resume_score:.4f}")


✅ **Real Resume Similarity Score:** 0.5963
🔹 **Real Resume Ranking Score:** 0.5963


In [ ]:
# Resume ranking based on job description
def rank_resumes(job_description, top_n=5):
    job_description_cleaned = clean_text(job_description)
    job_vector = vectorizer.transform([job_description_cleaned])
    
    # Use Nearest Neighbors to find similar resumes
    nn = NearestNeighbors(n_neighbors=top_n, metric='cosine')
    nn.fit(X)
    distances, indices = nn.kneighbors(job_vector)
    
    return df.iloc[indices[0]]

# Example usage
job_desc = "Looking for a data scientist with expertise in Python and machine learning."
top_resumes = rank_resumes(job_desc, top_n=5)
print(top_resumes[['category', 'resume_text']])

In [122]:
#final step above 